In [20]:
from queue import Queue
import time
import copy

In [21]:
grid = "003020600900305001001806400008102900700000008006708200002609500800203009005010300"

In [22]:
def inicializar(cadena: str):
    if len(cadena) != 81 :
        raise ValueError("La cadena no tiene 81 caracteres")
    
    matriz = []

    for i in range (9):
        fila = []
        for j in range (9):
            fila.append(int(cadena[j+i*9]))
        matriz.append(fila)

    return matriz

In [23]:
mat = inicializar(grid)

In [24]:
class Problema:

    def __init__(self, estado_inicial, celda_por_filas=False):
        self.estado_inicial = estado_inicial
        self.estrategia = celda_por_filas

    def posiblesValores(self, tablero, fila, columna):
        """
        Obtiene los poribles valores de una celda respetando las reglas del Sudoku.

        tablero: el estado actual del tablero
        fila: la posicion de fila de la celda
        columna: la posicion de columna de la celda
        """
        # comprueba que la delda esta no esta ocupada
        if tablero[fila][columna] != 0:
            return []

        # inicializa todos los valores posibles
        valores_posibles = set(range(1, 10))

        # Elimina los valores ya usados en la misma fila y columna
        for k in range(9):
            valores_posibles.discard(tablero[fila][k])
            valores_posibles.discard(tablero[k][columna])

        # Elimina los numeros ya usados en mismo cuadrante
        fila_cuadrante, columna_cuadrante = 3 * (fila // 3), 3 * (columna // 3)
        for i in range(3):
            for j in range(3):
                valores_posibles.discard(
                    tablero[fila_cuadrante + i][columna_cuadrante + j]
                )
        return valores_posibles

    def siguienteCelda(self, tablero):
        """
        Obtiene la siguiente celda a explorar dado un tablero. Dependiendo de self.estrategia hay dos opciones de busqueda:
            1: por filas de izquierda a derecha
            2: elige la celda que tenga el minimo numero de valores posibles

        tablero : estado actual del tablero
        """

        # Si buscamos la celda siguiente por la estrategia "1"
        if self.estrategia:
            celda = None
            for fila in range(9): 
                for columna in range(9):
                    if tablero[fila][columna] == 0:
                        posibles_valores = self.posiblesValores(tablero, fila, columna)
                        celda = (fila, columna, posibles_valores)
            return celda

        # Si buscamos la celda siguiente mediante la estatregia "2"
        else:
            min_valores = 10  # inicializamos con 10 ya que el numero maximo de valores posibles es 9
            mejor_celda = None

            for fila in range(9):
                for columna in range(9):
                    if tablero[fila][columna] == 0:
                        posibles_valores = self.posiblesValores(tablero, fila, columna)
                        if (
                            len(posibles_valores)
                            < min_valores  # evaluamos si la celda actual es mejor que la previa
                            and len(posibles_valores) > 0
                        ):
                            min_valores = len(posibles_valores)
                            mejor_celda = (fila, columna, posibles_valores)

            return mejor_celda

    def resultadoAccion(self, estado, accion):
        """
        Devuelve el estado del tablero despues de aplicarle la accion.

        estado: el estado del tablero en ese momento.
        accion: array que contiene [fila, columna, valor]
        """
        fila = accion[0]
        columna = accion[1]
        valor = accion[2]

        # Add new valid value to board
        nuevo_estado = copy.deepcopy(estado)
        nuevo_estado[fila][columna] = valor

        return nuevo_estado

    def esFinal(self, estado):
        """
        comprueba si el estado actual es el estado final
        estado: el estado actual del tablero

        return: True si el estado es el final, False si no lo es
        """
        # Inicializa la suma esperada de cada fila, columna y cuadrante para se considerado final
        total = sum(range(1, 9 + 1))

        # Comprueba las filas y columans
        for fila in range(9):
            if (len(estado[fila]) != 9) or (sum(estado[fila]) != total):
                return False

            columna_total = 0
            for columna in range(9):
                columna_total += estado[columna][fila]

            if columna_total != total:
                return False

        # Comprueba los cuadrantes
        for columna in range(0, 9, 3):
            for fila in range(0, 9, 3):

                total_cuadrante = 0
                for fila_cuadrante in range(0, 3):
                    for columna_cuadrante in range(0, 3):
                        total_cuadrante += estado[fila + fila_cuadrante][
                            columna + columna_cuadrante
                        ]

                if total_cuadrante != total:
                    return False

        return True

In [25]:
class Nodo: 

    def __init__(self, estado, accion=None):
        self.estado = estado
        self.accion = accion
        

    def expandir(self, problema):
        """
        aplica los valores posibles a la celda

        problema: el problema a solucionar
        """
        celda = problema.siguienteCelda(self.estado)
        self.fila = celda[0]
        self.columna = celda[1]
        self.acciones = celda[2]
        
        return [self.nodoHijo(problema, accion)
                for accion in self.acciones]

    def nodoHijo(self, problema, accion):
        """
        crea un nodo hijo dado el estado actual y una accion
        """
        celda_actualizada = [self.fila, self.columna, accion]
        siguienteNodo = problema.resultadoAccion(self.estado, celda_actualizada)
        return Nodo(siguienteNodo, celda_actualizada)


In [26]:
def busquedaAnchura(problema):
    """
    Dado un tablero inicializado realiza la busqueda en anchura
    """
    #guardamos el estado inicial en un nodo
    nodo  = Nodo(problema.estado_inicial)

    # comprobamos si el estado inicial es el final
    if problema.esFinal(nodo.estado):
        return nodo
    
    # inicialiamos la frontera
    frontera = Queue()
    frontera.put(nodo)

    while (frontera.qsize() != 0):

        # obtenemos los nodos de la cola de la frontera de manera FIFO
        nodo = frontera.get()
        
        for hijo in nodo.expandir(problema):
            if problema.esFinal(hijo.estado):
                return hijo
            
            frontera.put(hijo)

    return None

    


In [27]:
def solucionar(board, estrategia):

    if estrategia:
        print ("\nResolviendo utilizando la estrategia de seleccion de celda por filas...")
    else: 
        print ("\nResolviendo utilizando la estrategia de seleccion de celda por numero minimo de valores posibles...")

    problema = Problema(board, celda_por_filas = estrategia)

    imprimirSudoku(problema.estado_inicial)
    ti = time.time()

    solucion = busquedaAnchura(problema)
    delta_tiempo = time.time() - ti

    if solucion:
        # for row in solution.estado:
        #     print (row)
        imprimirSudoku(solucion.estado)
    else:
        print ("No tiene solucion")

    print ("Ha tardado: " + str(delta_tiempo) + " segundos")

In [28]:
solucionar(mat, estrategia=True)        


Resolviendo utilizando la estrategia de seleccion de celda por filas...
0 0 3 | 0 2 0 | 6 0 0
9 0 0 | 3 0 5 | 0 0 1
0 0 1 | 8 0 6 | 4 0 0
---------------------
0 0 8 | 1 0 2 | 9 0 0
7 0 0 | 0 0 0 | 0 0 8
0 0 6 | 7 0 8 | 2 0 0
---------------------
0 0 2 | 6 0 9 | 5 0 0
8 0 0 | 2 0 3 | 0 0 9
0 0 5 | 0 1 0 | 3 0 0



4 8 3 | 9 2 1 | 6 5 7
9 6 7 | 3 4 5 | 8 2 1
2 5 1 | 8 7 6 | 4 9 3
---------------------
5 4 8 | 1 3 2 | 9 7 6
7 2 9 | 5 6 4 | 1 3 8
1 3 6 | 7 9 8 | 2 4 5
---------------------
3 7 2 | 6 8 9 | 5 1 4
8 1 4 | 2 5 3 | 7 6 9
6 9 5 | 4 1 7 | 3 8 2



Ha tardado: 0.17856168746948242 segundos


In [29]:
solucionar(mat, estrategia=False)


Resolviendo utilizando la estrategia de seleccion de celda por numero minimo de valores posibles...
0 0 3 | 0 2 0 | 6 0 0
9 0 0 | 3 0 5 | 0 0 1
0 0 1 | 8 0 6 | 4 0 0
---------------------
0 0 8 | 1 0 2 | 9 0 0
7 0 0 | 0 0 0 | 0 0 8
0 0 6 | 7 0 8 | 2 0 0
---------------------
0 0 2 | 6 0 9 | 5 0 0
8 0 0 | 2 0 3 | 0 0 9
0 0 5 | 0 1 0 | 3 0 0



4 8 3 | 9 2 1 | 6 5 7
9 6 7 | 3 4 5 | 8 2 1
2 5 1 | 8 7 6 | 4 9 3
---------------------
5 4 8 | 1 3 2 | 9 7 6
7 2 9 | 5 6 4 | 1 3 8
1 3 6 | 7 9 8 | 2 4 5
---------------------
3 7 2 | 6 8 9 | 5 1 4
8 1 4 | 2 5 3 | 7 6 9
6 9 5 | 4 1 7 | 3 8 2



Ha tardado: 0.010498762130737305 segundos


In [15]:
def imprimirSudoku(tablero):
    """Imprime el tablero."""
    for i, fila in enumerate(tablero):
        if i % 3 == 0 and i != 0:
            print(
                "-" * 21
            )  
        print(
            " ".join(f"{num}" for num in fila[:3])
            + " | "
            + " ".join(f"{num }" for num in fila[3:6])
            + " | "
            + " ".join(f"{num }" for num in fila[6:])
        )

    print("\n\n")

